#VoiceRecognition wav file exploration and understanding

This notebook is to understand the data format and layout of recording using Python Wave, wav files, and numpy arrays all representing the same underlying audio

##1) Record some audio to investigate with

###Prep
Import what we need

In [32]:
import wave
import numpy as np
import struct
from array import array
import sys
import pyaudio

###Config for the stream

In [33]:
THRESHOLD = 150  # Originally 500, but new is_silent() works better with this value
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 44100

RECORD_SECONDS = 5

###Set up and record a stream from the mic


In [68]:
p = pyaudio.PyAudio()
stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK_SIZE * RECORD_SECONDS)):
    data = stream.read(CHUNK_SIZE)
    frames.append(data) # 2 bytes(16 bits) per channel
    # print len(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

* recording
* done recording


###Display some information on what we've recorded

*len(data) will show us how much data is captured durectly from stream.read() based on our CHUNK_SIZE
*len(frames) will show us how much data has been recorded directly into our frames buffer

In [69]:
print "Data recorded for each stream.read(): ", len(data)
print "Total number of frames captured in frames: ", len(frames)


Data recorded for each stream.read():  2048
Total number of frames captured in frames:  215


In [70]:
print "Number of CHUNKs recorded in total:", len(data) * len(frames)

Number of CHUNKs recorded in total: 440320


Now display some of the data that we've captured to get a flavour of what it looks like

In [71]:
print "Data from stream.read():"
print data
print "\nData in frames:"
print frames[:2]

Data from stream.read():
� A e � � � � v � � � { u � � � � � � � � � � f w � � � � � � � � � � � � � � � �  � � � � (� � � � � w l � � � � � � � � � � � � � � � � � � #� � � � � � � � g [ � � l 7 V q g 2 ������  ��������������M�<�l�����$��_���l�.���#�>�[�F�������'���������
���w�����������a�R�u�����f�,�,�f�������q�m����������������3���������8�6�B�_�W�H�d�o�I�8�I�\�]�N�P�_�a�]�f�}�k�D�M�k�i�h��������������������� # )   ! ^ { W V q W & * y � � J R � � � i W 6 ( = X � � ] 8 n � S ����C � � = 2 _ j ^ ` g K  5 � � w %  w � � .  \ � � � Z K Z w � � � r _ z � � u l  � � � t ? X � � � � � � � � � � � .� � 	?� � DB� 7-$-3+� � 
� � � 
&.� � � � � � � � � � � � e Y S Q � � X (   
 S i  ���� % ����������������l�m�������~�Q�9�q�����9�����M�_�1���������7�;���������������������������q�g��������������������������������������������������������������?�/��$�S�n�\�T�~���^�T�������p�G�r�����������������������p��� ����t����� �����

###Write this file to disk for later

In [72]:
wf = wave.open("testing.wav", 'wb')
wf.setnchannels(1)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes('b'.join(frames))
wf.close()

###Using Numpy
Let's view some of this data in numpy as that's have we're going to manipulate it.

1) First we need to convert the frames data from integer into string (or can we get around this without as the stream.read() call returns a string in data???

In [75]:
fs = ",".join(str(f) for f in frames)
r = np.array(struct.unpack_from(
        "%dh" % len(frames)*CHUNK_SIZE, 
        fs))

# Now print some stats about what we've just created in np
print r.shape
print r

(220160,)
[ 0  0  0 ..., 47 23 34]
